In [65]:
from typing import List, Tuple
import pandas as pd
import altair as alt
import random

alt.data_transformers.enable('json')


DataTransformerRegistry.enable('json')

## Check if points are on a curve

Code for example 4-1

In [66]:
a, b = 0, 7

p = 115792089237316195423570985008687907853269984665640564039457584007908834671663
x = 49790390825249384486033144355916864607616083520101638681403973749255924539515
y = 59574132161899900045862086493921015780032175291755807399284007721050341297360
print((x ** 3 + x*a + b - y**2) % p)

0


In [67]:
def _is_on_curve(
    x: int, 
    y: int, 
    p: int,
    a: int = 0,
    b: int = 7,
):
    return ((x ** 3 + x*a + b - y**2) % p) == 0

In [68]:
_is_on_curve(x, y, p)

True

## Exploring the elliptic curves a little bit

In [69]:
p = 17
w = 200
h = 200


points: List[Tuple[int, int]] = []
for x in range(w):
    for y in range(h):
        if _is_on_curve(x, y, p):
            points.append((x, y))

print(len(points))

2377


In [70]:
df = pd.DataFrame(points, columns=["x", "y"])

p1 = points[random.randint(0, len(points) - 1)]
p2 = points[random.randint(0, len(points) - 1)]

base_points_chart = alt.Chart(df).mark_circle().encode(
    x="x",
    y="y"
)

chosen_df = pd.DataFrame([p1, p2], columns=["x", "y"])
chosen_points_chart = alt.Chart(chosen_df).mark_line(color="red").encode(
    x="x",
    y="y"
)

(chosen_points_chart + base_points_chart).interactive()

alt.LayerChart(...)

## Trying out some ECC maths

A public key is just a point on the elliptic curve that equals $k*G$, where k is the private key and G is the 
"generator point" associated with the curve.

I just want to try calculate G + G, or the public key for k=1

In [71]:
from ecc import find_y, ecc_addition

In [72]:
points = []

a, b, p = 0, 7, 37
for x in range(100):
    try: 
        y = find_y(x, p)
        if y != 0:
            points.append((x, y))
            points.append((x, -y))
    except Exception:
        pass


In [73]:
alt.Chart(pd.DataFrame(points, columns=["x", "y"])).mark_circle(color="red").encode(
    x="x",
    y="y"
).interactive()

alt.Chart(...)

In [74]:
p1, p2 = points[0], points[6]

p3 = ecc_addition(p1, p2, p)

In [75]:
_is_on_curve(*p3, p)

True

## Playing with the private keys from the book

In [76]:
k = 0xf8f8a2f43c8376ccb0871305060d7b27b0554d2cc72bccf41b2705608452f315
g = (
    0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798,
    0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8
)
p = 115792089237316195423570985008687907853269984665640564039457584007908834671663

ecc_addition(g, g, p)

(5659563192761508084413547218350839200768777758085375688457209287130601213183,
 83121579216557378445487899878180864668798711284981320763518679672151497189239)

In [77]:
_is_on_curve(*g, p)

True

In [78]:
_is_on_curve(*ecc_addition(g, g, p), p)

False

That's a bit disapointing. I suspect that we are running into precision errors.

# Debugging using known libs

In [79]:
import ecdsa

In [80]:
g = (
    ecdsa.SECP256k1.generator.x(),
    ecdsa.SECP256k1.generator.y()
)
p = ecdsa.SECP256k1.curve.p()
# Note this is also easily accessible from ecdsa.SECP256k1.curve
curve = ecdsa.ellipticcurve.CurveFp(p, a, b)

In [81]:
p_g = ecdsa.ellipticcurve.Point(curve, *g)
p2 = p_g + p_g
_is_on_curve(p2.x(), p2.y(), curve.p(), curve.a(), curve.b())

True

In [82]:
ecc_addition(g, g, p)

(5659563192761508084413547218350839200768777758085375688457209287130601213183,
 83121579216557378445487899878180864668798711284981320763518679672151497189239)

In [83]:
p2.x(), p2.y()

(89565891926547004231252920425935692360644145829622209833684329913297188986597,
 12158399299693830322967808612713398636155367887041628176798871954788371653930)

Oh well, back to the book.

# Generating a public key

In [84]:
# private key from the book
k = 0xf8f8a2f43c8376ccb0871305060d7b27b0554d2cc72bccf41b2705608452f315

In [85]:
curve = ecdsa.SECP256k1.curve
G = ecdsa.SECP256k1.generator

K = k * G

In [86]:
# same as the book, nice.
hex(K.x()), hex(K.y())

('0x6e145ccef1033dea239875dd00dfb4fee6e3348b84985c92f103444683bae07b',
 '0x83b5c38e5e2b0c8529d7fa3f64d46daa1ece2d9ac14cab9477d042c84c32ccd0')

In [87]:
# Uncompressed point SEC1 serialisation of public key
pk = "04" + hex(K.x())[2:] + hex(K.y())[2:]
pk

'046e145ccef1033dea239875dd00dfb4fee6e3348b84985c92f103444683bae07b83b5c38e5e2b0c8529d7fa3f64d46daa1ece2d9ac14cab9477d042c84c32ccd0'

# Hashing

In [88]:
from hashlib import sha3_256


In [89]:
h = sha3_256()

In [90]:
h.update(b"")

In [91]:
h.hexdigest()

'a7ffc6f8bf1ed76651c14756a061d662f580ff4de43b49fa82d80a4b80f8434a'

In [92]:
# This is not Keccak. I'll probably have to use a non-standard lib

In [93]:
from Crypto.Hash import keccak

k = keccak.new(digest_bits=256)
k.update(b"")
k.hexdigest()

'c5d2460186f7233c927e7db2dcc703c0e500b653ca82273b7bfad8045d85a470'

# Generate Ethereum Address

In [ ]:
import codecs

In [100]:
# get key in correct format
K_string = hex(K.x())[2:] + hex(K.y())[2:]
K_bytes = codecs.decode(K_string, "hex")

# hash
kec = keccak.new(digest_bits=256)
kec.update(K_bytes)
addr = kec.hexdigest()[-40:]
addr

'001d3f1ef827552ae1114027bd3ecf1f086ba0f9'

# EIP-55 Encoding

In [114]:
def eip55_encode(addr: str):
    addr_hash_kec = keccak.new(digest_bits=256)

    # NOTE: it's the utf-8 encoding of the lower case address
    addr_hash_kec.update(codecs.encode(addr.lower(), 'utf-8'))
    addr_hash = addr_hash_kec.hexdigest()

    encoded_addr = ""
    for a, h in zip(addr, addr_hash):
        encoded_addr += a.upper() if int(h, base=16) >= 8 else a

    return encoded_addr

In [118]:
encoded_addr = eip55_encode(addr)
encoded_addr

'001d3F1ef827552Ae1114027BD3ECF1f086bA0F9'

In [120]:
# make a typo
letters = list(encoded_addr)
letters[5] = "4"
typo_addr = "".join(letters)

In [121]:
eip55_encode(typo_addr.lower()) == typo_addr

False

In [123]:
eip55_encode(encoded_addr.lower()) == encoded_addr

True

In [2]:
hex(10000000000000000)

'0x2386f26fc10000'